## Non Compliance Experiment=1
Test top norms for different w_nc

In [24]:
import sys
sys.path.append('../src')

In [146]:
import yaml
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
from mcmc_norm_learning.algorithm_1_v4 import to_tuple

In [2]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

In [6]:
%%time
with io.capture_output() as exp_nc_0_log:
    !python ../scripts/nc_experiments.py -exp exp_nc0 -w_nc 0.0 -n_threads 6
write_log(exp_nc_0_log.stdout,"../data_nc/exp_nc0/exp_nc_0_run.log")  

CPU times: user 7.88 s, sys: 2.16 s, total: 10 s
Wall time: 28min 3s


In [7]:
!grep "Time for" ../data_nc/exp_nc0/exp_nc_0_run.log

In [8]:
!grep "log" ../data_nc/exp_nc0/exp_nc_0_run.log

In [ ]:
!dvc add ../data_nc/exp_nc0/
!git add --all
!git commit -m "Results gen for dummy_exp_nc_0"
!dvc push

### Loop over w_nc values

In [9]:
w_nc_list=[0.0,0.05,0.1,0.2,0.3,0.4]

In [10]:
for w_nc in tqdm(w_nc_list,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    with io.capture_output() as exp_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 6
    write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name)) 

## Analysis
### 1. True Norm log_posterior for diff w_nc

In [55]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    prior_out=!grep "For True Norm" ../data_nc/{exp_dir_name}/run.log
    post_out=!grep "log_post_true_norm" ../data_nc/{exp_dir_name}/run.log
    print (f"For w_nc={w_nc}:")
    print (prior_out)
    print (post_out)
    print()

For w_nc=0.0:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-12769.387404068873']

For w_nc=0.05:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13097.518410107548']

For w_nc=0.1:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13146.65914171823']

For w_nc=0.2:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13187.900778087323']

For w_nc=0.3:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13428.957121845662']

For w_nc=0.4:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_

### 2. True Norm atleast mentioned in any chains, for diff w_nc

In [19]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    out=!grep "True norm in some chain(s)" ../data_nc/{exp_dir_name}/chain_info.txt
    print (f"For w_nc={w_nc}, True Norm found={out}")

For w_nc=0.0, True Norm found=['True norm in some chain(s): True']
For w_nc=0.05, True Norm found=['True norm in some chain(s): True']
For w_nc=0.1, True Norm found=['True norm in some chain(s): True']
For w_nc=0.2, True Norm found=['True norm in some chain(s): True']
For w_nc=0.3, True Norm found=['True norm in some chain(s): False']
For w_nc=0.4, True Norm found=['True norm in some chain(s): False']


### 3. Rank of True Norm as per log_post, iff it was found

In [28]:
with open("../params_nc.yaml", 'r') as fd:
    params = yaml.safe_load(fd)
true_norm=to_tuple(params['true_norm']['exp'])

In [57]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    rank_df=pd.read_csv(f"../data_nc/{exp_dir_name}/ranked_posteriors.csv",index_col=False)
    rank_post=rank_df.loc[rank_df.expression==str(true_norm)][["post_rank","log_posterior"]].values
    if rank_post.shape[0]==1:
        print (f"For w_nc={w_nc}, True Norm rank={rank_post[0][0]},wi posterior={rank_post[0][1]}")

For w_nc=0.0, True Norm rank=1.0,wi posterior=-12783.31601471459
For w_nc=0.05, True Norm rank=1.0,wi posterior=-13111.447020753265
For w_nc=0.1, True Norm rank=1.0,wi posterior=-13160.587752363943
For w_nc=0.2, True Norm rank=1.0,wi posterior=-13201.82938873304


#### ==> True Norm always has highest posterior, when its found,
confirm if it is in one chain, or multiple

### 4. In how many chains, max post crosses post(true_norm)

In [59]:
stacked=pd.DataFrame()
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    result = pd.read_csv(f"../data_nc/{exp_dir_name}/chain_posteriors_nc.csv")
    result["w_nc"] = w_nc
    stacked=stacked.append(result)

In [63]:
grouped=stacked.groupby(["w_nc","chain_number"])[["log_posterior"]].agg(['min', 'max', 'mean', 'std'])

In [168]:
true_post={
    "a":-12784,"b":-13112,\
    "c":-13161,"d":-13202,\
    "e":1,"f":1
}

In [169]:
grouped["w_nc"]=[x[0] for x in grouped.index]
grouped["key"]=np.where(grouped["w_nc"]==0.0,"a",\
                        np.where(grouped["w_nc"]==0.05,"b",\
                                np.where(grouped["w_nc"]==0.1,"c",\
                                        np.where(grouped["w_nc"]==0.2,"d",\
                                                np.where(grouped["w_nc"]==0.0,"e","f")\
                                                )\
                                        )\
                                )\
                        )

In [170]:
grouped.style.apply(lambda x: ['background: lightgreen' if x['log_posterior',  'max'] >=\
                               true_post[x["key"].values.item()] else '' for i in x], 
               axis=1)

### 5. If True Norm, features in Top Norms

In [227]:
top_norms=stacked.groupby(["w_nc","expression"]\
               )[["chain_number","log_posterior"\
                 ]].agg({"log_posterior":'mean',"chain_number":"count"}\
                       ).sort_values(by=["chain_number"]\
                                    ).groupby(["w_nc"\
                                              ]).tail(20).sort_values(by=["w_nc","chain_number"],\
                                                                     ascending=[True, False])

In [228]:
top_norms["grp_rnk"]=top_norms.groupby("w_nc")[["chain_number"]].rank(method='dense',ascending=False)

In [229]:
top_norms.loc[0.0].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["a"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'g'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'circle'], ['PerZone', '1']]]",-13555.627969,1459,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'g'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'r'], ['Shape', 'any'], ['PerZone', '1']]]",-13555.627969,1342,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'square'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '2']]]",-13584.396176,1184,3.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '2']]]",-13198.105711,1053,4.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'r'], ['Shape', 'any']]], ['Zone', '2']]]",-13358.889864,841,5.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'r'], ['Shape', 'any']]], ['Zone', '2']]]",-13358.889864,779,6.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '2']]]",-13198.105711,727,7.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-12783.316015,676,8.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'circle']]], ['Zone', '2']]]",-13358.889864,648,9.000000


In [232]:
top_norms.loc[0.05].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["b"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13111.447021,3509,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13111.447021,2413,2.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]], ['Zone', '3']]]",-13520.542021,1411,3.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13112.545633,1060,4.000000


In [234]:
top_norms.loc[0.1].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["c"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13160.587752,4246,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13160.587752,4125,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'square'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '1']]]",-13620.195162,2654,3.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13161.686365,1385,4.000000


In [236]:
top_norms.loc[0.2].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["d"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13201.829389,2860,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13201.829389,2353,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-13491.769002,2229,3.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'circle'], ['PerZone', '1']]]",-13618.816501,1803,4.000000


In [237]:
top_norms.loc[0.3].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["e"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'triangle'], ['PerZone', '1']]]",-13475.004794,1758,1.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'r'], ['Shape', 'any']]]], ['Zone', '3']]]",-13499.673654,1672,2.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'circle']]]], ['Zone', '3']]]",-13499.673654,1657,3.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'any'], ['Shape', 'square'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]]], ['Zone', '2']]]",-13659.440810,990,4.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'b'], ['Shape', 'triangle'], ['PerZone', '1']]]",-13476.103406,836,5.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '2']]]",-13570.795324,660,6.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'r'], ['Shape', 'circle']]]], ['Zone', '3']]]",-13500.772266,567,7.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'b'], ['Shape', 'any'], ['PerZone', 'any']]]",-13631.687112,529,8.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'square'], ['Zone', '2'], ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '2']]]",-13571.893937,428,9.000000
